In [19]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import json
import matplotlib.pyplot as plt
import plotly.express as px
import imageio
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read in the data from the raw Zillow CSV for ZALL
zillow_path = Path("../data/clean_data/zillow_data_houston_sa_austin_dallas.csv")
salary_path = Path()
zillow_df = pd.read_csv(zillow_path, infer_datetime_format=True, parse_dates=True)

zillow_df["date"] = pd.to_datetime(zillow_df["date"])
zillow_df.head()

,date,region,avg_value,zipcode,city
0,1996-01-31,75201; TX; Dallas-Fort Worth-Arlington; Dallas...,412559.0,75201,Dallas
1,1996-02-29,75201; TX; Dallas-Fort Worth-Arlington; Dallas...,413722.0,75201,Dallas
2,1996-03-31,75201; TX; Dallas-Fort Worth-Arlington; Dallas...,416215.0,75201,Dallas
3,1996-04-30,75201; TX; Dallas-Fort Worth-Arlington; Dallas...,422319.0,75201,Dallas
4,1996-05-31,75201; TX; Dallas-Fort Worth-Arlington; Dallas...,428461.0,75201,Dallas


In [3]:
#Read in the salary data from the IRS csv
salary_df = pd.read_csv("../data/clean_data/cnc_sal_data.csv", infer_datetime_format= True, parse_dates= True)
salary_df["Avg_AGI"] = salary_df["AGI"] / salary_df["NO. OF RETURNS"] * 1000
salary_df = salary_df.dropna()
salary_df.head()

,YEAR,ZIP CODE,NO. OF RETURNS,AGI,Avg_AGI
0,2007,73301,1798,7230,4021.134594
1,2007,73949,43,1915,44534.883721
2,2007,75001,7128,693056,97230.078563
3,2007,75002,26669,2002247,75077.693202
4,2007,75006,21953,1053748,48000.182207


In [4]:
#Clean up the salary Dataframe because we no longer need a couple of columns
salary_df = salary_df.drop(columns=["AGI","NO. OF RETURNS"])
salary_df.head()

,YEAR,ZIP CODE,Avg_AGI
0,2007,73301,4021.134594
1,2007,73949,44534.883721
2,2007,75001,97230.078563
3,2007,75002,75077.693202
4,2007,75006,48000.182207


In [5]:
#Clean up the Zillow CSV to Group By Year to get average house value by zip code by year, then rename the columns to match the salary data
zillow_year_df = zillow_df.groupby(["zipcode","city",zillow_df["date"].dt.year]).mean()
zillow_year_df = zillow_year_df.reset_index()
zillow_year_df = zillow_year_df.rename(columns={"date" : "YEAR", "zipcode" : "ZIP CODE", "city":"CITY"})
zillow_year_df.head()

,ZIP CODE,CITY,YEAR,avg_value
0,75201,Dallas,1996,425752.833333
1,75201,Dallas,1997,434680.833333
2,75201,Dallas,1998,432947.333333
3,75201,Dallas,1999,436265.250000
4,75201,Dallas,2000,436422.583333


In [10]:
#Merge the Zillow data and the IRS data by year and zip code, then create a column for Opportunity Ratio
combined_df = pd.merge(zillow_year_df, salary_df, on=['YEAR', 'ZIP CODE'])
combined_df["OP_RATIO"] = combined_df['Avg_AGI'] / combined_df["avg_value"]
combined_df.head()

,ZIP CODE,CITY,YEAR,avg_value,Avg_AGI,OP_RATIO
0,75201,Dallas,2007,590771.500000,402407.690759,0.681156
1,75201,Dallas,2008,610801.750000,271227.125119,0.444051
2,75201,Dallas,2009,571372.750000,190436.672968,0.333297
3,75201,Dallas,2010,547731.166667,192694.852396,0.351806
4,75201,Dallas,2011,521611.583333,192270.074877,0.368608


In [15]:
houston_2018_df = combined_df[(combined_df["CITY"] == "Houston") & (combined_df["YEAR"]==2018)]
houston_df.head()

,ZIP CODE,CITY,YEAR,avg_value,Avg_AGI,OP_RATIO
900,77002,Houston,2007,207800.333333,243247.477745,1.170583
901,77002,Houston,2008,202157.000000,350087.246457,1.731759
902,77002,Houston,2009,179833.666667,256497.117400,1.426302
903,77002,Houston,2010,169905.333333,505074.139676,2.972680
904,77002,Houston,2011,165413.083333,349702.761947,2.114118


In [20]:
#Import the TX zipcode GeoGSON file
with open('../data/tx_texas_zip_codes_geo.min.json') as f:
    tx_zip = json.load(f)